# 6.2. Случайный лес

Ваша задача - написать класс `random_forest` для решения задачи классификации на основе датасета Ирисов Фишера (`sklearn.datasets.load_iris`), принимающий на вход конструктора аргументы `n_estimators`, `max_depth`, `subspaces_dim` и `random_state`. описание этих аргументов приведено ниже. У этого класса должны быть определены методы `.fit()` и `.predict()`, а также поле `._estimators`, в котором должен храниться список алгоритмов, используемых в ансамбле. 

Для создания обучающей подвыборки для каждого из базовых классификаторов, Вы можете воспользоваться классом `sample`, который Вы реализовали в прошлом задании. В случае его использования, не забудьте включить его описание в файл с Вашим решением текущего задания. Мы также предлагаем вам организовать выбор подпространств для каждого дерева посредством заполнения списка `subspace_idx`, в котором будут логироваться выбранные для каждого базового классификатора подпространства.

Замечание: в рамках выполнения данного задания запрещено использовать класс `sklearn.ensemble.RandomForestClassifier`. Такой код не пройдёт проверку.

Подберите также гиперпараметры, на которых ваш алгоритм получает наилучшее качество (с точки зрения метрики accuracy, доли правильных ответов) на тестовой выборке с параметром `test_size`=0.3, задайте их в виде глобальных переменных N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM.

Шаблон класса:

In [79]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
np.random.seed(42)

class sample(object):
    def __init__(self, X, n_subspace):
        self.idx_subspace = self.random_subspace(X, n_subspace)
    
    def __call__(self, X, y):
        idx_obj = self.bootstrap_sample(X)
        X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
        return X_sampled, y_sampled

    @staticmethod
    def bootstrap_sample(X, random_state=42):
        return np.unique(np.random.choice(X.shape[0], X.shape[0]))

    @staticmethod
    def random_subspace(X, n_subspace, random_state=42):
        return np.random.choice(X.shape[1], n_subspace, replace=False)

    @staticmethod
    def get_subsample(X, y, idx_subspace, idx_obj):
        X_sampled = X[idx_obj][:, idx_subspace]
        y_sampled = y[idx_obj]
        return X_sampled, y_sampled

N_ESTIMATORS = 3
MAX_DEPTH = 3
SUBSPACE_DIM = 3

class random_forest(object):
    def __init__(self, n_estimators: int, max_depth: int, subspaces_dim: int, random_state: int):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subspaces_dim = subspaces_dim
        self.random_state = random_state
        self._estimators = []
        self.subspace_idx = []

    def fit(self, X, y):
        for i in range(self.n_estimators):
            s = sample(X, self.subspaces_dim)
            dt = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
            X_sampled, y_sampled = s(X, y)
            dt.fit(X_sampled, y_sampled)
            self._estimators.append(dt)
            self.subspace_idx.append(s.idx_subspace)

    def predict(self, X):
        predicted = np.zeros((X.shape[0],))
        for i in range(self.n_estimators):
            predicted += self._estimators[i].predict(X[:, self.subspace_idx[i]])

        return (predicted / self.n_estimators).astype(int)

In [152]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rf = random_forest(3, 3, 3, 42)
rf.fit(X_train, y_train)
predictions = rf.predict(X_test)
accuracy_tree = accuracy_score(predictions, y_test)
print(accuracy_tree)
# mas = []
# for i in range (1, 100):
#     for j in range (1, 100):
#         for k in range(1, 4):
#             rf = random_forest(i, j, k, 42)
#             rf.fit(X_train, y_train)
#             predictions = rf.predict(X_test)
#             accuracy_tree = accuracy_score(predictions, y_test)
#             mas.append([i, j, k, accuracy_tree])

# index = 0
# max = 0
# for i in range (len(mas)):
#     if (mas[i][3] > max):
#         max = mas[i][3]
#         index = i

# print(mas[i][0], mas[i][1], mas[i][2], mas[i][3])

0.9555555555555556


## Примечания

1. В данной задаче запрещено использовать библиотеку pandas.

2. В реализуемых методах запрещается использовать вывод любой информации на экран (в частности, недопустимо использование print()).

# Пример входных и выходных данных

In [ ]:
X = np.array([[ 0.47819456, -1.57891216, -0.1018819 ,  1.11113501,  0.20826281,
        -1.11091227,  0.07844205],
       [ 0.11850997,  1.91073022,  0.95574903,  1.35798262,  0.56177995,
         0.26012021,  0.42404407],
       [-0.52304666,  0.75051167, -1.037804  , -0.10105312,  0.08559063,
         0.5102743 , -1.79068927],
       [-0.09078024,  1.62097709,  0.93284371,  1.0386902 , -0.68354252,
        -1.27138661,  0.15060651],
       [ 0.11676701, -0.71769062, -0.80119565,  0.73448495,  1.80728052,
         0.45770337,  0.20689119]])

In [ ]:
X.shape

(5, 7)

In [ ]:
y = np.array([1, 0, 1, 1, 0])
y.shape

(5,)

In [ ]:
rf = random_forest(25, 15, 2, 42)
rf.fit(X, y)

In [ ]:
preds = rf.predict(X)

In [ ]:
preds

array([1, 0, 1, 1, 0])